In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [29]:
# Daily WEATHER data
w_daily = pd.read_csv("data/weather/daily.csv",encoding='euc-kr', sep=',')
w_daily.head(2)

,Location,Time,AvgTemp,MinTemp,MaxTemp
0,112,2016-07-20,28.1,24.6,31.6
1,112,2016-07-21,28.0,25.0,31.5


In [30]:
w_daily2 = w_daily.drop(columns='Location')
w_daily2.head(2)

,Time,AvgTemp,MinTemp,MaxTemp
0,2016-07-20,28.1,24.6,31.6
1,2016-07-21,28.0,25.0,31.5


In [31]:
w_daily2['Time'] = pd.to_datetime(w_daily2.Time)
w_daily2 = w_daily2.set_index('Time')
w_daily2.head(2)

,AvgTemp,MinTemp,MaxTemp
Time,,,
2016-07-20,28.1,24.6,31.6
2016-07-21,28.0,25.0,31.5


In [7]:
######################
# Monthly WEATHER data
w_Monthly = pd.read_csv("data/weather/monthly.csv",encoding='euc-kr', sep=',')
w_Monthly.head(2)

,Location,Time,AvgTemp,AvgMaxTemp,AvgMinTemp,MaxTemp,MinTemp
0,112,16-Aug,27.4,30.9,24.6,34.0,16.9
1,112,16-Sep,22.9,26.7,19.8,29.1,15.4


In [8]:
w_MonthlyTemp = w_Monthly.drop(columns=['Location', 'MaxTemp','MinTemp'])
w_Monthly2 = w_MonthlyTemp.iloc[12:,:]
w_Monthly2.head(2)

,Time,AvgTemp,AvgMaxTemp,AvgMinTemp
12,17-Aug,25.4,28.3,23.1
13,17-Sep,21.9,25.5,18.5


In [9]:
w_Monthly2['Mth'] = pd.DataFrame([8, 9, 10,11,12, 1,2,3,4,5,6,7,8,9,10,11]).values
w_Monthly2.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Time,AvgTemp,AvgMaxTemp,AvgMinTemp,Mth
12,17-Aug,25.4,28.3,23.1,8
13,17-Sep,21.9,25.5,18.5,9
14,17-Oct,15.7,19.9,12.1,10
15,17-Nov,5.9,9.9,2.3,11
16,17-Dec,-1.3,2.1,-4.7,12


In [10]:
w_Monthly2 = w_Monthly2.drop(columns='Time')
w_Monthly2

,AvgTemp,AvgMaxTemp,AvgMinTemp,Mth
12,25.4,28.3,23.1,8
13,21.9,25.5,18.5,9
14,15.7,19.9,12.1,10
15,5.9,9.9,2.3,11
16,-1.3,2.1,-4.7,12
17,-3.8,-0.6,-6.6,1
18,-1.8,2.0,-5.2,2
19,6.9,11.1,3.5,3
20,11.9,16.3,8.2,4
21,17.0,21.1,13.3,5


## 파생 및 dummy변수

In [15]:
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [16]:
test.index = pd.to_datetime(test.index)
test.index

DatetimeIndex(['2017-08-29 10:00:00', '2017-08-29 11:00:00',
               '2017-08-29 12:00:00', '2017-08-29 13:00:00',
               '2017-08-29 14:00:00', '2017-08-29 15:00:00',
               '2017-08-29 16:00:00', '2017-08-29 17:00:00',
               '2017-08-29 18:00:00', '2017-08-29 19:00:00',
               ...
               '2018-06-30 14:00:00', '2018-06-30 15:00:00',
               '2018-06-30 16:00:00', '2018-06-30 17:00:00',
               '2018-06-30 18:00:00', '2018-06-30 19:00:00',
               '2018-06-30 20:00:00', '2018-06-30 21:00:00',
               '2018-06-30 22:00:00', '2018-06-30 23:00:00'],
              dtype='datetime64[ns]', name='Time', length=7334, freq=None)

In [17]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [ ]:
# def Daily_dateTimeVar(df):
#     df1 = pd.DataFrame(df.index)
#     df1.index = pd.to_datetime(df1.iloc[:,0])
#     df1 = df1.drop(columns=df1.columns)
#     # datetime을 사용한 파생변수
#     # 연도, 월, 일, 요일, 시간
#     df1['Mth']= df1.index.month
#     df1['Date']=df1.index.day
#     df1['Day']=df1.index.weekday
# #     df1['hour']=df1.index.hour
    
#     # Season 1: 봄 ~ 4: 겨울 
#     df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
#                 1 if x in [3,4,5] else 2 if x in [6,7,8]
#                else 3)
#     # dayClass 1: 월, 2: 화~금, 3: 토, 일 
#     # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
#     df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
#                 2 if x in [1,2,3,4] else 3)
#     return df1

In [18]:
TE_D_time_vars = dateTimeVar(test)

In [19]:
TE_D_time_vars.head(1)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2


## Make DUMMIES

In [21]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [23]:
TE_D_timeV_dum = dummies(TE_D_time_vars, ['season', 'dayClass'])

In [24]:
TE_D_timeV_dum.head(2)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3
Time,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0


In [25]:
# TE_timeV_Daily_dum_wthr= pd.concat([TE_D_timeV_dum, w_daily2], axis=1,join='inner')

In [42]:
TE_D_timeV_dum['tmpDate'] = pd.to_datetime(TE_D_timeV_dum.index.date)

In [43]:
TE_D_timeV_dum.head(2)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,tmpDate
Time,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,2017-08-29
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0,2017-08-29


In [44]:
w_daily2.head(2)

,AvgTemp,MinTemp,MaxTemp
Time,,,
2016-07-20,28.1,24.6,31.6
2016-07-21,28.0,25.0,31.5


## Feature DF (daily, monthly)

In [47]:
dFeatures = pd.merge(TE_D_timeV_dum, w_daily2, left_on = 'tmpDate',right_index=True, how='inner').drop(columns='tmpDate')

# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [48]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [49]:
dFeatures.head(1)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,AvgTemp,MinTemp,MaxTemp
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,18.9,17.1,22.2


<br>

**칼럼 10~50개 돌리기**

In [ ]:
##################################################
#### 칼럼 바꾸기 #################################

test2 = test_D.iloc[:,:50]
test2.head()

## X_Test (prediction) 생성

In [52]:
submission_daily = pd.date_range(start = pd.datetime(2018, 7, 1, 0),
                             end = pd.datetime(2018, 7, 10, 23),
                             freq = "H")
submission_daily

DatetimeIndex(['2018-07-01 00:00:00', '2018-07-01 01:00:00',
               '2018-07-01 02:00:00', '2018-07-01 03:00:00',
               '2018-07-01 04:00:00', '2018-07-01 05:00:00',
               '2018-07-01 06:00:00', '2018-07-01 07:00:00',
               '2018-07-01 08:00:00', '2018-07-01 09:00:00',
               ...
               '2018-07-10 14:00:00', '2018-07-10 15:00:00',
               '2018-07-10 16:00:00', '2018-07-10 17:00:00',
               '2018-07-10 18:00:00', '2018-07-10 19:00:00',
               '2018-07-10 20:00:00', '2018-07-10 21:00:00',
               '2018-07-10 22:00:00', '2018-07-10 23:00:00'],
              dtype='datetime64[ns]', length=240, freq='H')

In [64]:
tmp_test = pd.DataFrame(index = submission_daily)
X_test_D = dummies(dateTimeVar(tmp_test), ['season', 'dayClass'])
X_test_D['season_1'] =0 ; X_test_D['season_3'] =0 ;X_test_D['season_4'] =0


X_test_D['tmpDate'] = pd.to_datetime(X_test_D.index.date)
X_test_D = pd.merge(X_test_D, w_daily2, left_on = 'tmpDate',right_index=True, how='inner').drop(columns='tmpDate')

In [65]:
X_test_D.head(2)

,Mth,Date,Day,hour,season_2,dayClass_1,dayClass_2,dayClass_3,season_1,season_3,season_4,AvgTemp,MinTemp,MaxTemp
0,,,,,,,,,,,,,,
2018-07-01 00:00:00,7,1,6,0,1,0,0,1,0,0,0,21.9,21.1,23.8
2018-07-01 01:00:00,7,1,6,1,1,0,0,1,0,0,0,21.9,21.1,23.8


In [68]:
X_test_D.shape

(240, 14)

In [ ]:
submission_monthly = pd.date_range(start = pd.datetime(2018, 7, 1),
                             end = pd.datetime(2018, 11, 30),
                             freq = "D")
submission_monthly

In [53]:
tmp_test = pd.DataFrame(index = submission_monthly)

X_test_M = dummies(Daily_dateTimeVar(tmp_test), ['season', 'dayClass'])
X_test_M['season_1'] =0 ; X_test_M['season_3'] =0 ;X_test_M['season_4'] =0
X_test_M['dayClass_1'] =0 ; X_test_M['dayClass_2'] =0 
X_test_M.head(2)

NameError: name 'submission_monthly' is not defined

In [ ]:
w_Monthly_T = w_Monthly2.tail(5)

In [ ]:
X_test_M = pd.merge(X_test_M, w_Monthly_T, left_on = 'Mth', right_on='Mth', how='left')

X_test_M = X_test_M.set_index(submission_monthly)
X_test_M.index.name = 'Time'
X_test_M.head()

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from lightgbm import LGBMRegressor

# lg_param = {"boosting_type":['gbdt', 'dart','goss'],
#     "learning_rate": [0.01,0.05,0.1,0.15],
#     "min_child_samples":[10, 20, 30, 50],       
#     "n_estimators":[10, 50, 100],
#     }

# def gsLGBR_Daily_FitPred (model, params, mID, X_train, y_train, X_test):
#     grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
#     grid_gb.fit(X_train, y_train)
#     fcst = grid_gb.predict(X_test)
#     a = pd.DataFrame([mID], columns=['meter_id'])
#     for i in range(10):
#         a['X2018_7_'+str(i+1)+'_d']=[fcst[i]]
#     return a

# def gsLGBR_Monthly_FitPred (model, params, mID, X_train, y_train, X_test):
#     grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
#     grid_gb.fit(X_train, y_train)
#     fcst = grid_gb.predict(X_test)
#     a = pd.DataFrame([mID], columns=['meter_id'])
    
#     a['X2018_7_m'] = [np.sum(fcst[:31])] # 7월 
#     a['X2018_8_m'] = [np.sum(fcst[31:62])] # 8월
#     a['X2018_9_m'] = [np.sum(fcst[62:92])] # 9월
#     a['X2018_10_m'] = [np.sum(fcst[92:123])] # 10월
#     a['X2018_11_m'] = [np.sum(fcst[123:153])]
    
#     return a
   
# def test50_Daily_GS_LGBReg(train, X_test_D, X_test_M):
#     daily = pd.DataFrame()
#     monthly = pd.DataFrame()
#     for mID in train.columns:
#         # Day
#         dataD = pd.concat([train.loc[:,[mID]], TE_timeV_Daily_dum_wthr], axis=1)
#         dataD = dataD.rename(columns={mID:'y'})
#         for i in [7]:
#             dataD['lag_{}'.format(i)] = dataD.y.shift(i)
#         y_trainD = dataD.dropna().y
#         X_trainD = dataD.dropna().drop(['y'], axis=1)
        
#         X_test_D['lag_7'] = np.append(y_train[292:299].values, y_trainD[289:292].values)
#         model = LGBMRegressor(verbose=1, tree_learner='feature')
#         tmpD = gsLGBR_Daily_FitPred(model, lg_param,
#                                     mID, X_trainD, y_trainD, X_test_D)
#         daily = pd.concat([daily, tmpD], axis=0)
        
#         ## Month
#         dataM = pd.concat([train.loc[:,[mID]], TE_timeV_Monthly_dum_wthr], axis=1)
#         dataM = dataM.rename(columns={mID:'y'})
#         y_trainM = dataM.dropna().y
#         X_trainM = dataM.dropna().drop(['y'], axis=1)        
        
#         model = LGBMRegressor(verbose=1, tree_learner='feature')
#         tmpM = gsLGBR_Monthly_FitPred(model, lg_param,
#                             mID,X_trainM, y_trainM,X_test_M)
#         monthly = pd.concat([monthly, tmpM], axis=0)
        
#     return  daily, monthly

In [109]:
from tqdm import tqdm_notebook
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[10, 50, 100],
    }

def gsLGBR_Daily_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    
    a['X2018_7_1d'] = [np.sum(fcst[:24])] # 7월 
    a['X2018_7_2d'] = [np.sum(fcst[24:48])] # 8월
    a['X2018_7_3d'] = [np.sum(fcst[48:72])] # 9월
    a['X2018_7_4d'] = [np.sum(fcst[72:96])] # 10월
    a['X2018_7_5d'] = [np.sum(fcst[96:120])]
    a['X2018_7_6d'] = [np.sum(fcst[120:144])]
    a['X2018_7_7d'] = [np.sum(fcst[144:168])]
    a['X2018_7_8d'] = [np.sum(fcst[168:192])]
    a['X2018_7_9d'] = [np.sum(fcst[192:216])]
    a['X2018_7_9d'] = [np.sum(fcst[216:240])]
    
    return a
   
def test50_Daily_fromHr_GS_LGBReg(train, X_test_D):
    hrly_50 = pd.DataFrame()
    for mID in tqdm_notebook(train.columns):
        data = pd.concat([train.loc[:,[mID]], dFeatures], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test_D['lag_168'] = np.append(y_train[-168:].values, y_train[-168:-96].values)
        
        model = LGBMRegressor(verbose=1, tree_learner='feature')
        tmp = gsLGBR_Daily_FitPred(model, lg_param,
                                  mID, X_train, y_train, X_test_D)
        hrly_50 = pd.concat([hrly_50, tmp], axis=0)
    return hrly_50

In [ ]:
daily = test50_Daily_fromHr_GS_LGBReg(test, X_test_D)

Fitting 4 folds for each of 144 candidates, totalling 576 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=10 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=10, score=0.065, total=   0.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=10 
[CV]  boosting_type=gbdt, learning_rate=0.01, min_child_samples=10, n_estimators=10, score=0.130, total=   0.0s


...

[CV] boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=10 
[CV]  boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=10, score=0.140, total=   0.0s
[CV] boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50, score=-0.083, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50, score=0.063, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50, score=0.226, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.05, min_child_samples=50, n_estimators=50, score=0.151, total=   0.1s
[CV] boosting_type=

[CV]  boosting_type=gbdt, learning_rate=0.1, min_child_samples=30, n_estimators=100, score=-0.128, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.1, min_child_samples=30, n_estimators=100 
[CV]  boosting_type=gbdt, learning_rate=0.1, min_child_samples=30, n_estimators=100, score=0.016, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.1, min_child_samples=30, n_estimators=100 
[CV]  boosting_type=gbdt, learning_rate=0.1, min_child_samples=30, n_estimators=100, score=0.251, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.1, min_child_samples=30, n_estimators=100 
[CV]  boosting_type=gbdt, learning_rate=0.1, min_child_samples=30, n_estimators=100, score=0.083, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.1, min_child_samples=50, n_estimators=10 
[CV]  boosting_type=gbdt, learning_rate=0.1, min_child_samples=50, n_estimators=10, score=-0.126, total=   0.0s
[CV] boosting_type=gbdt, learning_rate=0.1, min_child_samples=50, n_estimators=10 
[CV]  boosting_type=g

[CV] boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=10 
[CV]  boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=10, score=0.175, total=   0.0s
[CV] boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=10 
[CV]  boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=10, score=0.115, total=   0.0s
[CV] boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=50, score=-0.119, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=50, score=0.011, total=   0.1s
[CV] boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=50 
[CV]  boosting_type=gbdt, learning_rate=0.15, min_child_samples=30, n_estimators=50, score=0.217, total=   0.1s
[CV] boosting_type=

[CV]  boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=50, score=0.113, total=   0.1s
[CV] boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100 
[CV]  boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100, score=-0.613, total=   0.1s
[CV] boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100 
[CV]  boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100, score=-1.689, total=   0.1s
[CV] boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100 
[CV]  boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100, score=-3.087, total=   0.1s
[CV] boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100 
[CV]  boosting_type=dart, learning_rate=0.01, min_child_samples=20, n_estimators=100, score=-1.386, total=   0.1s
[CV] boosting_type=dart, learning_rate=0.01, min_child_samples=30, n_estimators=10 
[CV]  bo

[CV]  boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=10, score=-0.252, total=   0.0s
[CV] boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=10 
[CV]  boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=10, score=0.017, total=   0.0s
[CV] boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=10 
[CV]  boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=10, score=-0.095, total=   0.0s
[CV] boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=10 
[CV]  boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=10, score=0.104, total=   0.0s
[CV] boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=50 
[CV]  boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=50, score=0.076, total=   0.1s
[CV] boosting_type=dart, learning_rate=0.05, min_child_samples=20, n_estimators=50 
[CV]  boosting_typ

In [ ]:
test2.shape

In [ ]:
pred_D100, pred_M100 = test50_Daily_GS_LGBReg(test2, X_test_D,X_test_M)

In [ ]:
pred_D100.head()

In [ ]:
pred_M100.head()

In [ ]:
import os
pred_D100.to_csv(r'100_daily.csv')
pred_M100.to_csv(r'100_Monthy.csv')

from IPython.display import FileLink
FileLink(r'100_daily.csv')
FileLink(r'100_Monthy.csv')